<a href="https://colab.research.google.com/github/toanpt74/COLAB_RD/blob/main/VAE_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
tf.keras.backend.clear_session()
from glob import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np
import shutil

"""
Setup paprameters
"""
# tf.random.set_seed(42)

dir=r'E:\PIC\__RESIZE'
dir_predict =dir+"\Test_predict"


#
# re_dir=dir+"_Mapping"
# if not os.path.exists(re_dir):
#     os.makedirs(re_dir)
files = glob(dir+"\\*.bmp")
vae = tf.keras.models.load_model(r'G:\Toa\autoProj\SVI_REAR_221029\LINE_SCAN_VENH_RIGHT_L_496x32_221029\LINE_SCAN_VENH_RIGHT_L_496x32_25000__3.126014471054077')
COL = 496 #width
ROW = 32 #height

def checkpatch(split,display=True, name=None):

    RESULT=True     #--->OK
    threshold = 0.1
    x = split.astype('float32') / 255.0
    vae_img = x.reshape((1, COL, ROW, 1))

    x_recon = vae.predict(vae_img)
    x_recon=x_recon.reshape(ROW,COL)

    config = vae.get_config()  # Returns pretty much every information about your model
    print(config["layers"][0]["config"]["batch_input_shape"])

    print(vae_img.shape)
    print(x_recon.shape)
    feature_score = np.power(x-x_recon,2)
    outlier_map=np.where(feature_score<threshold,0,255)

    count_gray = np.count_nonzero(outlier_map)
    print("count_gray: ", count_gray)

    outlier_map255=outlier_map.astype('uint8')

    # kernel = np.ones((3, 3), np.uint8)
    # # dilate=cv2.dilate(outlier_map255,kernel)
    # # erode=cv2.erode(dilate,kernel)
    # morpho=cv2.morphologyEx(outlier_map255, cv2.MORPH_OPEN, kernel)
    _, thresh = cv2.threshold(outlier_map255, 20, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    outcnt = [cnt for cnt in contours if (cv2.contourArea(cnt) > 10)]

    #
    # # print(f'x={np.max(x)},,vae_img={np.max(vae_img)},x_recon={np.max(x_recon)},feature_map={np.max(feature_score)}')
    # # print(f'outlier_map255={np.max(outlier_map255)},morpho={np.max(morpho)},color_x={np.max(color_x)}')
    #
    if outcnt:
        RESULT = False

    # if name=='DOT1b':
    #     display=True
    if display:
        fig, axes = plt.subplots(2, 4, figsize=(8, 4))
        axes[0, 0].imshow(x*255, cmap='gray',vmin=0,vmax=255)
        axes[0, 0].set_title('Input image')
        axes[0, 1].imshow(x_recon*255, cmap='gray',vmin=0,vmax=255)
        axes[0, 1].set_title('Reconstruction')
        axes[0, 2].imshow(feature_score)
        axes[0, 2].set_title('Error')
        axes[0, 3].imshow(outlier_map)
        axes[0, 3].set_title('outlier map')

        # axes[1,0].imshow(outlier_map255)
        # axes[1,0].set_title('Error G255')
        # axes[1,1].imshow(morpho,cmap='gray')
        # axes[1,1].set_title('Morpho')
        # axes[1,2].imshow(thresh)
        # axes[1,2].set_title('Threshold')
        # axes[1, 3].imshow(color_x)
        # axes[1, 3].set_title('Mapping')
    plt.show()
    return RESULT

import time
for file in files:
    start=time.time()
    img = cv2.imread(file,0)
    # img=cv2.copyMakeBorder(img,0,1,0,1,borderType=cv2.BORDER_REPLICATE)
    # imgRGB=cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    # kernel=cv2.getStructuringElement(cv2.MORPH_RECT,ksize=(7,7))
    # img=cv2.dilate(img,kernel=kernel,iterations=3)
    # basename=os.path.basename(file)
    # print(basename)
    # name = basename.rsplit('.',1)[0]


    RE=checkpatch(img,display=False)

    if not os.path.exists(dir_predict):
        os.makedirs(dir_predict)

    if RE:
        path_save = os.path.join(dir_predict, "OK")
        if not os.path.exists(path_save):
            os.makedirs(path_save)
        shutil.copyfile(file, path_save + "\\" + file.split("\\")[-1])
    else:
        path_save = os.path.join(dir_predict, "NG")
        if not os.path.exists(path_save):
            os.makedirs(path_save)
        shutil.copyfile(file, path_save + "\\" + file.split("\\")[-1])

        # if not RE:
        #     cv2.rectangle(imgRGB, (w_size, h_size), (w_size + split_size, h_size + split_size), (0, 0, 255), 2)

    # os.chdir(re_dir)
    # cv2.imwrite(f'MAPPING_{basename}',imgRGB)
    print(f"Tact time: {time.time()-start}")
    tf.keras.backend.clear_session() ##importance

print('DONE')
